# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [17]:
# Dependencies and Setup

import hvplot.pandas
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import cartopy
import geoviews
import json

# Import API key
from api_keys import geoapify_key
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,puerto natales,-51.7236,-72.4875,6.69,90,99,1.61,CL,1697585751
1,1,aginskoye,51.1000,114.5300,1.35,76,77,6.39,RU,1697585752
2,2,adamstown,-25.0660,-130.1015,22.52,85,100,4.18,PN,1697585753
3,3,blackmans bay,-43.0167,147.3167,16.96,53,34,0.89,AU,1697585753
4,4,avarua,-21.2078,-159.7750,25.03,61,54,7.20,CK,1697585754


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [20]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    color = "City"
)
hvplot.save(map_plot_1, '../output_data/AllCities.html')
# Display the map
map_plot_1


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_df = city_data_df[ (city_data_df["Max Temp"] >= 22) &  (city_data_df["Humidity"] <= 60) ]
clean_ideal_city_df = ideal_df[ ideal_df["Cloudiness"] == 0 ] 

# Drop any rows with null values
clean_ideal_data_df = clean_ideal_city_df.dropna()

# Display sample data
clean_ideal_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
21,21,turabah,21.2141,41.6331,23.40,46,0,2.55,SA,1697585765
22,22,siwa oasis,29.2041,25.5195,22.48,57,0,2.30,EG,1697585766
26,26,tindouf,27.6711,-8.1474,26.25,25,0,4.12,DZ,1697585769
74,74,san patricio,28.0170,-97.5169,24.40,41,0,3.83,US,1697585802
112,112,crane,31.3974,-102.3501,24.09,26,0,7.37,US,1697585830
113,113,haiku-pauwela,20.9219,-156.3051,30.01,48,0,3.60,US,1697585831
146,146,ruidoso,33.3317,-105.6730,23.08,17,0,4.12,US,1697585854
152,152,cabo san lucas,22.8909,-109.9124,34.23,51,0,2.06,MX,1697585859
163,163,karratha,-20.7377,116.8463,31.13,17,0,3.06,AU,1697585867
196,196,lackland air force base,29.3866,-98.6180,26.04,26,0,3.09,US,1697585707


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
hotel_df = clean_ideal_data_df[['City','Country','Lat','Lng','Humidity']].copy()
hotel_df['Hotel Name'] = ""
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
21,turabah,SA,21.2141,41.6331,46,
22,siwa oasis,EG,29.2041,25.5195,57,
26,tindouf,DZ,27.6711,-8.1474,25,
74,san patricio,US,28.0170,-97.5169,41,
112,crane,US,31.3974,-102.3501,26,
113,haiku-pauwela,US,20.9219,-156.3051,48,
146,ruidoso,US,33.3317,-105.6730,17,
152,cabo san lucas,MX,22.8909,-109.9124,51,
163,karratha,AU,-20.7377,116.8463,17,
196,lackland air force base,US,29.3866,-98.6180,26,


In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = clean_ideal_data_df[['City','Country','Lat','Lng','Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Reset Index
newindex = np.arange(0,len(hotel_df))
hotel_df = hotel_df.set_index(newindex)

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,turabah,SA,21.2141,41.6331,46,
1,siwa oasis,EG,29.2041,25.5195,57,
2,tindouf,DZ,27.6711,-8.1474,25,
3,san patricio,US,28.0170,-97.5169,41,
4,crane,US,31.3974,-102.3501,26,
5,haiku-pauwela,US,20.9219,-156.3051,48,
6,ruidoso,US,33.3317,-105.6730,17,
7,cabo san lucas,MX,22.8909,-109.9124,51,
8,karratha,AU,-20.7377,116.8463,17,
9,lackland air force base,US,29.3866,-98.6180,26,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel

radius = 10000
categories = 'accommodation.hotel'
limit = 1
params = {
    "categories":categories,
    "limit":limit,
    "apiKey":geoapify_key    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    city_lat = row.loc['Lat']
    city_lng = row.loc['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    filter = f"circle:{city_lng},{city_lat},{radius}"
    bias = f"proximity:{city_lng},{city_lat}"
    params["filter"] = filter
    params["bias"] = bias
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
turabah - nearest hotel: No hotel found
siwa oasis - nearest hotel: فندق الكيلانى
tindouf - nearest hotel: محمد بوسبي
san patricio - nearest hotel: No hotel found
crane - nearest hotel: No hotel found
haiku-pauwela - nearest hotel: Inn At Mama's Fish House
ruidoso - nearest hotel: Hotel Ruidoso
cabo san lucas - nearest hotel: Comfort Rooms
karratha - nearest hotel: Karratha International Hotel
lackland air force base - nearest hotel: Gateway Villa
ma'rib - nearest hotel: No hotel found
laguna - nearest hotel: Holiday Inn Express & Suites
port isabel - nearest hotel: Queen Isabel Inn
kingman - nearest hotel: Home2 Suites by Hilton Kingman
tulare - nearest hotel: No hotel found
port hedland - nearest hotel: The Esplanade Hotel
medina - nearest hotel: The Oberoi Madina
jackson - nearest hotel: Brookside
winnemucca - nearest hotel: The Overland Hotel
winslow - nearest hotel: La Posada Hotel
prineville - nearest hotel: Best Western
marco island - nearest hotel: JW Marr

,City,Country,Lat,Lng,Humidity,Hotel Name
0,turabah,SA,21.2141,41.6331,46,No hotel found
1,siwa oasis,EG,29.2041,25.5195,57,فندق الكيلانى
2,tindouf,DZ,27.6711,-8.1474,25,محمد بوسبي
3,san patricio,US,28.0170,-97.5169,41,No hotel found
4,crane,US,31.3974,-102.3501,26,No hotel found
5,haiku-pauwela,US,20.9219,-156.3051,48,Inn At Mama's Fish House
6,ruidoso,US,33.3317,-105.6730,17,Hotel Ruidoso
7,cabo san lucas,MX,22.8909,-109.9124,51,Comfort Rooms
8,karratha,AU,-20.7377,116.8463,17,Karratha International Hotel
9,lackland air force base,US,29.3866,-98.6180,26,Gateway Villa


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [21]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    color = "City",
    hover_cols = ['City','Humidity','Hotel Name','Country']
)
hvplot.save(map_plot_2, '../output_data/Hotels.html')

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)